In [ ]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.layers import Dense

In [ ]:
Table_main = []
Table_adv = []
Data_main = []
Ans_main = []
Data_adv = []
Ans_adv = []
useless = []

types = {' покой': 0, 'сто': 0, 'мест': 0, 'stoy': 0, 'ход': 1, 'пеш': 1, 'шаг ': 1, '_шаг': 1, '_go': 1, 'бег': 2,
         'run': 2, 'вели': 3, 'ипед': 3, 'лестн': 4, 'под': 4, 'stai': 4, 'авто': 5, 'машин': 5, 'метро': 6,
         'электр': 7, 'самок': 8}

In [ ]:
def get_type(x):
    for act in types:
        if x.lower().find(act) != -1:
            return types[act]
    return -1


def str_to_float(x):
    x = str(x)
    x = x.replace(',', '.')
    return float(x)

In [ ]:
names = os.listdir("/Data")

for name in names:
    if name[-5:] != ".xlsx":
        sep = ','
        file = open("/Data/" + name)
        if file.readline().find(';') != -1:
            sep = ';'
        file.close()

        tbl = pd.read_csv("/Data/" + name, sep=sep)
        tbl.columns = ['Time', 'X', 'Y', 'Z', 'Acc']
        tbl['Time'] = tbl['Time'].apply(lambda x: str_to_float(x))
        tbl = tbl[(7 < tbl.Time) & (20 > tbl.Time)]
        for i in ['X', 'Y', 'Z', 'Acc']:
            tbl[i] = tbl[i].apply(lambda x: str_to_float(x))

        if tbl.shape[0] > 1000:
            fr = tbl.shape[0] // 1000
            tbl = tbl.iloc[tbl.index % fr == 0]
            tbl.index = range(tbl.shape[0])
            tbl = tbl.iloc[tbl.index < 1000]

            if get_type(name) == -1:
                useless.append(name)
            elif get_type(name) < 5:
                answer = [0] * 5
                Table_main.append(tbl)
                Data_main.append(np.array(tbl.Acc))
                answer[get_type(name)] = 1
                Ans_main.append(np.array(answer))
            else:
                answer = [0] * 4
                Table_adv.append(tbl)
                Data_adv.append(np.array(tbl.Acc))
                answer[get_type(name) - 5] = 1
                Ans_adv.append(np.array(answer))
    # else:
    # print(name)

Data_main = np.array(Data_main)
Ans_main = np.array(Ans_main)
Data_adv = np.array(Data_adv)
Ans_adv = np.array(Ans_adv)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [ ]:
model = keras.Sequential([
    Dense(128, input_shape=(1000,), activation='relu'),
    Dense(5, activation='softmax')
])

model.compile(optimizer=keras.optimizers.Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 128)               128128    
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 645       
Total params: 128,773
Trainable params: 128,773
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
history = model.fit(Data_main, Ans_main, epochs=5, batch_size=10, validation_split = 0.2)

Epoch 1/5
130/130 [==============================] - 1s 4ms/step - loss: 1.5256 - accuracy: 0.3002 - val_loss: 1.3574 - val_accuracy: 0.5938
Epoch 2/5
130/130 [==============================] - 0s 2ms/step - loss: 1.3256 - accuracy: 0.4637 - val_loss: 1.3559 - val_accuracy: 0.5323
Epoch 3/5
130/130 [==============================] - 0s 3ms/step - loss: 1.2679 - accuracy: 0.4938 - val_loss: 1.3031 - val_accuracy: 0.3046
Epoch 4/5
130/130 [==============================] - 0s 2ms/step - loss: 1.1658 - accuracy: 0.5486 - val_loss: 1.2558 - val_accuracy: 0.5692
Epoch 5/5
130/130 [==============================] - 0s 2ms/step - loss: 1.1202 - accuracy: 0.5687 - val_loss: 1.2520 - val_accuracy: 0.5354


In [ ]:
model.save('Model_main')

INFO:tensorflow:Assets written to: model3/assets


In [ ]:
TestTbl_main = []
TestTbl_adv = []
Test_main = []
Test_adv = []
adv = [32, 34, 38, 46, 56, 59, 71, 101, 113, 142, 143]

for i in range(146):
    if i == 12 or i == 20 or i == 54 or i == 104:
        TestTbl_main.append('err')
        TestTbl_adv.append('err')
        Test_main.append(np.array([1]))
        Test_adv.append(np.array([1]))
        continue

    sep = ','
    file = open("/Test/track_" + str(i) + ".csv")
    if file.readline().find(';') != -1:
        sep = ';'
    file.close()

    tbl = pd.read_csv("/Test/track_" + str(i) + ".csv", sep=sep)
    tbl.columns = ['Time', 'X', 'Y', 'Z', 'Acc']
    tbl['Time'] = tbl['Time'].apply(lambda x: str_to_float(x))
    tbl = tbl[(7 < tbl.Time) & (20 > tbl.Time)]
    for j in ['X', 'Y', 'Z', 'Acc']:
        tbl[j] = tbl[j].apply(lambda x: str_to_float(x))

    if tbl.shape[0] > 1000:
        fr = tbl.shape[0] // 1000
        tbl = tbl.iloc[tbl.index % fr == 0]
        tbl.index = range(tbl.shape[0])
        tbl = tbl.iloc[tbl.index < 1000]
        if i not in adv:
            TestTbl_main.append(tbl)
            TestTbl_adv.append('adv')
            Test_main.append(np.array(tbl.Acc))
            Test_adv.append(np.array([0]))
        else:
            TestTbl_adv.append(tbl)
            TestTbl_main.append('main')
            Test_adv.append(np.array(tbl.Acc))
            Test_main.append(np.array([0]))
    else:
        TestTbl_main.append('err')
        TestTbl_adv.append('err')
        Test_main.append(np.array([1]))
        Test_adv.append(np.array([1]))

In [ ]:
model_main = keras.models.load_model('Model_main')
model_adv = keras.models.load_model('Model_adv')

In [ ]:
answers = [0] * 146

In [ ]:
def vec_to_ans(x):
    max = 0
    ans = 0
    for i in range(len(x[0])):
        if x[0][i] > max:
            ans = i
            max = x[0][i]
    return ans

In [ ]:
for i in range(146):
    if Test_main[i].shape[0] == 1 and Test_main[i][0] == 1:
        answers[i] = 2
    elif Test_main[i].shape[0] == 1 and Test_main[i][0] == 0:
        answers[i] = vec_to_ans(model_adv.predict(np.expand_dims(Test_adv[i], axis=0))) + 5
    else:
        answers[i] = vec_to_ans(model_main.predict(np.expand_dims(Test_main[i], axis=0)))

In [ ]:
df = pd.DataFrame({'track_num': range(146), 'action': answers})
df.to_csv('ans.csv', index=False)